In [1]:
import _gan_keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import umap
import seaborn as sns
from sklearn.decomposition import PCA
import copy
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn import metrics
from sklearn.metrics import balanced_accuracy_score
from sklearn import decomposition

1.13.1


# Reading the data

In [2]:

input_ltpm_matrix = pd.read_csv('pbmc_input.csv', delimiter=',')

In [3]:
input_ltpm_matrix_labs = pd.read_csv('pbmc_labels.csv', delimiter=',')
input_ltpm_matrix_states = pd.read_csv('pbmc_pert_labels.csv', delimiter=',')

In [4]:
input_ltpm_matrix_states.head()

,0
0,1
1,1
2,1
3,1
4,1


In [5]:
input_ltpm_matrix.head()

,AL627309.1,RP11-206L10.9,LINC00115,NOC2L,KLHL17,HES4,ISG15,TNFRSF18,TNFRSF4,SDF4,...,C21orf67,FAM207A,ADARB1,POFUT2,COL18A1,SLC19A1,COL6A2,FTCD,DIP2A,S100B
0,0.0,0.0,0.0,0.0,0.0,0.0,3.206646,0.947689,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,3.314060,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,2.344877,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,2.292093,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,2.430965,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
input_ltpm_matrix_labs.head()

,0
0,4
1,6
2,0
3,2
4,0


# Filter the data if needed

In [7]:
df_style = copy.deepcopy(input_ltpm_matrix)
df_style['labels'] = copy.deepcopy(input_ltpm_matrix_labs)
df_style['states'] = copy.deepcopy(input_ltpm_matrix_states)
df_style['id'] = df_style.labels.map(str) + " | " + df_style.states.map(str)


labs_filt = df_style['labels']
states_filt = df_style['states']
id_filt = df_style['id']

df_style = df_style.drop('labels', 1)
df_style = df_style.drop('states', 1)
df_style = df_style.drop('id', 1)


In [8]:
df_style.shape

(16893, 6998)

# RF

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
df_style, id_filt, test_size=0.5, random_state=42)

In [ ]:
clf = RandomForestClassifier(random_state=42)

In [ ]:
clf.fit(X_train, y_train)

/home/paureel/.local/lib/python3.5/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [ ]:
y_pred=clf.predict(X_test)

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Accuracy:",metrics.balanced_accuracy_score(y_test, y_pred))

Accuracy: 0.7860778974783947
Accuracy: 0.634090317880221


pca_pre = decomposition.PCA(n_components=7)
df_style_pca = pca_pre.fit_transform(df_style)
X_train, X_test, y_train, y_test = train_test_split(
df_style_pca, id_filt, test_size=0.5, random_state=42)
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Accuracy:",metrics.balanced_accuracy_score(y_test, y_pred))

In [ ]:
i = 0
acc = []
for index in range(1,100):    
    pca_pre = decomposition.PCA(n_components=100)
    df_style_pca = pca_pre.fit_transform(df_style)
    reducer = umap.UMAP(n_components = index)
    df_style_umapdata = reducer.fit_transform(df_style_pca)
    X_train, X_test, y_train, y_test = train_test_split(
    df_style_umapdata, id_filt, test_size=0.5, random_state=42)
    clf = RandomForestClassifier(random_state=42)
    clf.fit(X_train, y_train)
    y_pred=clf.predict(X_test)
    print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
    print("Accuracy:",metrics.balanced_accuracy_score(y_test, y_pred))
    acc.append(metrics.accuracy_score(y_test, y_pred))
    print(i)
    i += 1
    

/home/paureel/.local/lib/python3.5/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy: 0.7468923878299988
Accuracy: 0.6639814047240822
0


/home/paureel/.local/lib/python3.5/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy: 0.8111755652894519
Accuracy: 0.7242418279646463
1


/home/paureel/.local/lib/python3.5/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


...

Accuracy: 0.8323665206582218
Accuracy: 0.7518907460070876
85


/home/paureel/.local/lib/python3.5/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy: 0.8317745945306025
Accuracy: 0.750469206010644
86


/home/paureel/.local/lib/python3.5/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy: 0.832248135432698
Accuracy: 0.7525636058417902
87


In [ ]:
bardata_x = ['Raw','PCA-100','UMAP-20']
bardata_y = [0.786, 0.818, 0.838]

bardata_bal_x = ['Raw','PCA-100','UMAP-20']
bardata_bal_y = [0.634, 0.700, 0.750]

In [ ]:
fig, ax = plt.subplots()

bar1 = ax.bar(bardata_x, bardata_y)

plt.title('Comparison of pre-processing methods')
plt.ylabel('Accuracy')
plt.xlabel('Method')
plt.show()

In [ ]:
fig, ax = plt.subplots()

bar1 = ax.bar(bardata_bal_x, bardata_bal_y)

plt.title('Comparison of pre-processing methods')
plt.ylabel('Balanced accuracy')
plt.xlabel('Method')
plt.show()

# Train the weights on all the data

In [ ]:
GAN_test = _gan_keras.GAN_keras(np.amax(pd.DataFrame(X_train).values), X_train.shape[1], pd.DataFrame(X_train))

In [ ]:
GAN_test.plot_UMAP(200)

In [ ]:
GAN_test.train_all(150000)


In [ ]:
embedding_list = GAN_test.plot_UMAP(200)

In [ ]:
colors = embedding_list[2]
palette = np.array(sns.color_palette("hls", 2))
f = plt.figure(figsize=(8, 8))
ax = plt.subplot(aspect='equal')
scatter1 = ax.scatter(embedding_list[0][np.where(colors == 0)], 
                      embedding_list[1][np.where(colors == 0)], lw=0, s=5, label='Original')
scatter2 = ax.scatter(embedding_list[0][np.where(colors == 1)], 
                      embedding_list[1][np.where(colors == 1)], lw=0, s=5, label='Generated')


plt.xlim(-25, 25)
plt.ylim(-25, 25)

plt.xlabel('UMAP 1', fontsize=18)
plt.ylabel('UMAP 2', fontsize=18)
ax.axis('tight')
plt.legend(handles=[scatter1, scatter2])
plt.show()

In [ ]:
gendict = {}
gen_labs = []
gen_labs_col = []
i = 0
j = 0
for index in np.unique(y_train):
    
    X_train_f = copy.deepcopy(X_train)
    X_train_f['id'] = y_train
    labeliter = index
    X_train_f = X_train_f.loc[X_train_f['id'] == labeliter]
    X_train_f = X_train_f.drop('id', axis = 1)
    for index2 in range(len(X_train_f)):
        latentrepvec = GAN_test.recover_latentrep(X_train_f.iloc[0,:], 1000)
        gencell = GAN_test.draw_cells_with_seed(latentrepvec)
        gendict[i] = gencell.values[0]
        gen_labs.append(labeliter)
        gen_labs_col.append(j)
        i += 1
        if i % 100 == 0:
            print(str((i/len(X_train_f))*100) + "%")
    print(j)
    j +=1
    

In [ ]:
rfgendata = pd.DataFrame.from_dict(gendict, orient='index')

In [ ]:
rfgendata.to_csv('genout.csv')

In [ ]:
pca = PCA(n_components=100)
data_pca = pca.fit_transform(rfgendata)
reducer = umap.UMAP()
embedding = reducer.fit_transform(data_pca)
colors = np.array(gen_labs_col)
palette = np.array(sns.color_palette("hls", 14))
f = plt.figure(figsize=(8, 8))
ax = plt.subplot(aspect='equal')
sc = ax.scatter(embedding[:,0], embedding[:,1], lw=0, s=5,c=palette[colors.astype(np.int)])
plt.xlim(-25, 25)
plt.ylim(-25, 25)
plt.xlabel('UMAP 1', fontsize=18)
plt.ylabel('UMAP 2', fontsize=18)
ax.axis('tight')
plt.show()

In [ ]:
X_train_new = copy.deepcopy(X_train)
X_train_new = X_train_new.T.reset_index(drop=True).T.reset_index(drop=True)
X_train_new = X_train_new.append(rfgendata)

In [ ]:
y_train_new = copy.deepcopy(y_train)
y_train_new = pd.concat([y_train_new, pd.Series(gen_labs)])

In [ ]:
clf_aug = RandomForestClassifier(random_state=42)

In [ ]:
clf_aug.fit(X_train_new, y_train_new)

In [ ]:
y_pred=clf_aug.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
clf_wo = RandomForestClassifier(random_state=42)

In [ ]:
clf_wo.fit(rfgendata, pd.Series(gen_labs))

In [ ]:
y_pred=clf_wo.predict(X_test)
print("Accuracy:",balanced_accuracy_score(y_test, y_pred))

In [ ]:
clf_umap = RandomForestClassifier(random_state=42)
clf_umap.fit(embedding, pd.Series(gen_labs))

In [ ]:

data_pca = pca.transform(X_test)

embedding = reducer.transform(data_pca)

In [ ]:
y_pred=clf_umap.predict(embedding)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

# Interpolation

In [ ]:
def plot_start(test1, test2, df_filtered, df_filtered_start, df_filtered_end):
    gencell = GAN_test.draw_cells_with_seed(test1)
    gencell = gencell.append(GAN_test.draw_cells_with_seed(test2), ignore_index=True)
    gencell['lab'] = 2

    df_interp_orig = copy.deepcopy(df_filtered)
    df_1 =  df_filtered_start.reset_index(drop=True).iloc[df_filtered_start.reset_index(drop=True).index==1]
    df_2 = df_filtered_end.reset_index(drop=True).iloc[df_filtered_end.reset_index(drop=True).index==1]
    df_3 = df_1.append(df_2,ignore_index=True)
    df_3 = df_3.T.reset_index().T.reset_index(drop=True).iloc[1:]
    df_3['lab'] = 1
    df_interp_orig = df_interp_orig.T.reset_index().T.reset_index(drop=True).iloc[1:]
    df_interp_orig['lab'] = 0
    df_new = df_interp_orig.append(df_3,ignore_index=True)

    df_new = df_new.append(gencell,ignore_index=True)

    pca = PCA(n_components=100)
    data_pca = pca.fit_transform(df_new.loc[:, df_new.columns != 'lab'])
    reducer = umap.UMAP()
    embedding = reducer.fit_transform(data_pca)
    colors = df_new.loc[:,'lab']
    palette = np.array(sns.color_palette("hls", 3))
    f = plt.figure(figsize=(8, 8))
    ax = plt.subplot(aspect='equal')
    sc = ax.scatter(embedding[:,0], embedding[:,1], lw=0, s=40,c=palette[colors.astype(np.int)])
    plt.xlim(-25, 25)
    plt.ylim(-25, 25)

    ax.axis('tight')
    plt.show()

In [ ]:
def plot_interp(interpolated, df_filtered):
    
    df = copy.deepcopy(interpolated.reset_index(drop=True))
    df_orig = copy.deepcopy(df_filtered.loc[:, df_filtered.columns != 'lab'])
    df_orig = df_orig.T.reset_index().T.reset_index(drop=True).iloc[1:]
    df = df.T.reset_index().T.reset_index(drop=True).iloc[1:]
    df = df.assign(lab = 1)
    df_orig = df_orig.assign(lab = 0)
    df_new = df_orig.append(df, ignore_index=True)

    pca = PCA(n_components=100)
    data_pca = pca.fit_transform(df_orig.loc[:, df_orig.columns != 'lab'])
    
    data_pca_tr = pca.transform(df.loc[:, df.columns != 'lab'])
    
    reducer = umap.UMAP()
    
    embedding = reducer.fit_transform(data_pca)
    embedding_tr = reducer.transform(data_pca_tr)
    
    embedding_final = np.concatenate((embedding, embedding_tr), axis=0)
    
    colors = df_new.loc[:,'lab']
    palette = np.array(sns.color_palette("hls", 2))
    f = plt.figure(figsize=(8, 8))
    ax = plt.subplot(aspect='equal')
    sc = ax.scatter(embedding_final[:,0], embedding_final[:,1], lw=0, s=40,c=palette[colors.astype(np.int)])
    plt.xlim(-25, 25)
    plt.ylim(-25, 25)

    ax.axis('tight')
    plt.show()

In [ ]:
def plot_interp_gene(interpolated, df_filtered):
    
    df = copy.deepcopy(interpolated.reset_index(drop=True))
    df_orig = copy.deepcopy(df_filtered.loc[:, df_filtered.columns != 'lab'])
    df_orig = df_orig.T.reset_index().T.reset_index(drop=True).iloc[1:]
    df = df.T.reset_index().T.reset_index(drop=True).iloc[1:]
    df = df.assign(lab = 1)
    df_orig = df_orig.assign(lab = 0)
    df_new = df_orig.append(df, ignore_index=True)

    
    colors = df_new.loc[:,'lab']
    palette = np.array(sns.color_palette("hls", 2))
    f = plt.figure(figsize=(8, 8))
    ax = plt.subplot(aspect='equal')
    sc = ax.scatter(df_new.iloc[:,80], df_new.iloc[:,55], lw=0, s=40,c=palette[colors.astype(np.int)])
    plt.xlim(-25, 25)
    plt.ylim(-25, 25)

    ax.axis('tight')
    plt.show()

In [ ]:
df_1 =  copy.deepcopy(df_filtered_start.reset_index(drop=True).iloc[df_filtered_start.reset_index(drop=True).index==1])
df_2 = copy.deepcopy(df_filtered_end.reset_index(drop=True).iloc[df_filtered_end.reset_index(drop=True).index==1])
    
test1 = GAN_test.recover_latentrep(df_1.values[0], 100000)
test2 = GAN_test.recover_latentrep(df_2.values[0], 100000)



In [ ]:
test1_pd = pd.DataFrame(test1).T
test2_pd = pd.DataFrame(test2).T

In [ ]:
np.abs(df_1-df_2).max(axis=1)

In [ ]:
np.abs(test1_pd-test2_pd).max(axis=1)

In [ ]:
np.abs(df_1-df_2).apply(lambda x: pd.Series(np.concatenate([x.nlargest(2).index.values])), axis=1)


In [ ]:
np.abs(test1_pd-test2_pd).apply(lambda x: pd.Series(np.concatenate([x.nlargest(2).index.values])), axis=1)

In [ ]:
plot_start(test1, test2, df_filtered)

In [ ]:
for index in range(100):
    interpolated = GAN_test.interpolate_cells(test1, test2, index)
    plot_interp(interpolated, df_filtered)

# Style transfer

In [ ]:
#df_all = copy.deepcopy(input_ltpm_matrix)
#df_all['labels'] = input_ltpm_matrix_labs
#df_all['states'] = input_ltpm_matrix_states
#pertvec = GAN_test.train_pertvec(input_ltpm_matrix,input_ltpm_matrix_labs, input_ltpm_matrix_states,10000, 10, 10000)



In [ ]:
pert_vecs_mean = []

for index in range(6):
    print(index)
    pert_vecs_raw = {}
    sample_count = 50
    num_count = 10000

    df_transfer = copy.deepcopy(df_style)
    df_transfer['labels'] = labs_filt
    df_transfer['states'] = states_filt

    df_transfer = df_transfer[(df_transfer['labels']==index)]
    df_transfer_start = df_transfer[(df_transfer['states']==0)]
    df_transfer_end = df_transfer[(df_transfer['states']==1)]
    df_transfer_start = df_transfer_start.drop('labels', 1)
    df_transfer_start = df_transfer_start.drop('states', 1)
    df_transfer_end = df_transfer_end.drop('labels', 1)
    df_transfer_end = df_transfer_end.drop('states', 1)

    i = 0
    for index in range(10): # do it with random sampling

        if i % 1 == 0:
            print(str((i/sample_count)*100) + '%')
        i += 1
        startvec = df_transfer_start.iloc[index,:]
        endvec = df_transfer_end.iloc[index,:]

        start_noise = GAN_test.recover_latentrep(startvec, num_count)
        end_noise = GAN_test.recover_latentrep(endvec, num_count)

        pert_vecs_raw[index] =  end_noise - start_noise

    pert_vecs_mean.append(pd.DataFrame(pert_vecs_raw).median(axis=1))


In [ ]:
agg_means = pd.DataFrame(pert_vecs_mean).T.median(axis=1)

In [ ]:
#sns.pairplot(pd.DataFrame(pert_vecs_mean).T)

In [ ]:
true_starters = []
sim_ends = []
alpha_factor = 5
for j in range(50):
    labels_to_test = 6
    df_style_test = copy.deepcopy(df_style)
    df_style_test['labels'] = labs_filt
    df_style_test['states'] = states_filt

    df_style_test = df_style_test[(df_style_test['labels']==labels_to_test)]
    df_style_test_start = df_style_test[(df_style_test['states']==0)]
    df_style_test_end = df_style_test[(df_style_test['states']==1)]

    df_style_test = df_style_test.drop('labels', 1)
    df_style_test = df_style_test.drop('states', 1)
    df_style_test_start = df_style_test_start.drop('labels', 1)
    df_style_test_start = df_style_test_start.drop('states', 1)
    df_style_test_end = df_style_test_end.drop('labels', 1)
    df_style_test_end = df_style_test_end.drop('states', 1)

    df_1 =  copy.deepcopy(df_style_test_start.reset_index(drop=True).iloc[df_style_test_start.reset_index(drop=True).index==j])
    df_2 = copy.deepcopy(df_style_test_end.reset_index(drop=True).iloc[df_style_test_end.reset_index(drop=True).index==j])


    test1 = GAN_test.recover_latentrep(df_1.values[0], 100000)
    true_starters.append(df_1.values[0])
    test2 = np.array(test1 + alpha_factor * agg_means)
    sim_ends.append(GAN_test.draw_cells_with_seed(test2))
    print(j)
    #plot_start(test1, test2, df_style, df_style_test_start, df_style_test_end)

In [ ]:
true_starters

In [ ]:
plot_start(test1, test2, df_style, df_style_test_start, df_style_test_end)


In [ ]:
df_filtered_oob = copy.deepcopy(input_ltpm_matrix)
df_filtered_oob['labels'] = input_ltpm_matrix_labs
df_filtered_oob['states'] = input_ltpm_matrix_states

df_filtered_oob = df_filtered_oob[(df_filtered_oob['labels']==6)]
df_filtered_start_oob = df_filtered_oob[(df_filtered_oob['states']==0)]
df_filtered_end_oob = df_filtered_oob[(df_filtered_oob['states']==1)]

df_filtered_oob = df_filtered_oob.drop('labels', 1)
df_filtered_oob = df_filtered_oob.drop('states', 1)
df_filtered_start_oob = df_filtered_start_oob.drop('labels', 1)
df_filtered_start_oob = df_filtered_start_oob.drop('states', 1)
df_filtered_end_oob = df_filtered_end_oob.drop('labels', 1)
df_filtered_end_oob = df_filtered_end_oob.drop('states', 1)


In [ ]:
simdatadf = pd.DataFrame(sim_ends[0])
for index in range(len(sim_ends)-1):
    simdatadf=simdatadf.append(sim_ends[index+1])

oob = copy.deepcopy(df_filtered_oob)
true_starters_pd = pd.DataFrame(true_starters)
sim_ends_pd = pd.DataFrame(simdatadf)
oob = oob.T.reset_index().T.reset_index(drop=True).iloc[1:]
oob = oob.assign(lab = 0)
true_starters_pd = true_starters_pd.assign(lab = 1)
sim_ends_pd = sim_ends_pd.assign(lab = 2)

oob_all = copy.deepcopy(oob)
oob_all = oob_all.append(true_starters_pd)
oob_all = oob_all.append(sim_ends_pd)

generated_oob = true_starters_pd.append(sim_ends_pd, ignore_index=True)


pca = PCA(n_components=100)
data_pca = pca.fit_transform(oob.loc[:, oob.columns != 'lab'])
data_pca_gen = pca.transform(generated_oob.loc[:, generated_oob.columns != 'lab'])
#data_pca_tr = pca.transform(df.loc[:, df.columns != 'lab'])

#data_pca_tr = pca.transform(true_starters_pd)

reducer = umap.UMAP()

embedding = reducer.fit_transform(data_pca)
embedding_gen = reducer.transform(data_pca_gen)

embedding = np.concatenate((embedding, embedding_gen), axis=0)

embedding_tr = copy.deepcopy(pd.DataFrame(embedding))
embedding_tr['lab'] = oob_all.lab.reset_index(drop=True)

embedding_tr_start = embedding_tr[(embedding_tr['lab']==1)]
embedding_tr_end = embedding_tr[(embedding_tr['lab']==2)]
#embedding_tr = reducer.transform(data_pca_tr)

#embedding_final = np.concatenate((embedding, embedding_tr), axis=0)

colors = oob_all.loc[:,'lab']
palette = np.array(sns.color_palette("hls", 3))
f = plt.figure(figsize=(8, 8))
ax = plt.subplot(aspect='equal')
sc = ax.scatter(embedding[:,0], embedding[:,1], lw=0, s=40,c=palette[colors.astype(np.int)])
for index in range(len(embedding_tr_start)):
    plt.arrow(embedding_tr_start.iloc[index,0], embedding_tr_start.iloc[index,1], 
                  embedding_tr_end.iloc[index,0] - embedding_tr_start.iloc[index,0], 
              embedding_tr_end.iloc[index,1] - embedding_tr_start.iloc[index,1]
                 )

#sc = ax.scatter(embedding[:,0], embedding[:,1], lw=0, s=40)
plt.xlim(-25, 25)
plt.ylim(-25, 25)

ax.axis('tight')
plt.show()

In [ ]:
simdatadf = pd.DataFrame(sim_ends[0])
for index in range(len(sim_ends)-1):
    simdatadf=simdatadf.append(sim_ends[index+1])

oob = copy.deepcopy(input_ltpm_matrix)
true_starters_pd = pd.DataFrame(true_starters)
sim_ends_pd = pd.DataFrame(simdatadf)
oob = oob.T.reset_index().T.reset_index(drop=True).iloc[1:]
oob = oob.assign(lab = 0)
true_starters_pd = true_starters_pd.assign(lab = 1)
sim_ends_pd = sim_ends_pd.assign(lab = 2)

oob_all = copy.deepcopy(oob)
oob_all = oob_all.append(true_starters_pd)
oob_all = oob_all.append(sim_ends_pd)

generated_oob = true_starters_pd.append(sim_ends_pd, ignore_index=True)


pca = PCA(n_components=100)
data_pca = pca.fit_transform(oob.loc[:, oob.columns != 'lab'])
data_pca_gen = pca.transform(generated_oob.loc[:, generated_oob.columns != 'lab'])
#data_pca_tr = pca.transform(df.loc[:, df.columns != 'lab'])

#data_pca_tr = pca.transform(true_starters_pd)

reducer = umap.UMAP()

embedding = reducer.fit_transform(data_pca)
embedding_gen = reducer.transform(data_pca_gen)

embedding = np.concatenate((embedding, embedding_gen), axis=0)

embedding_tr = copy.deepcopy(pd.DataFrame(embedding))
embedding_tr['lab'] = oob_all.lab.reset_index(drop=True)

embedding_tr_start = embedding_tr[(embedding_tr['lab']==1)]
embedding_tr_end = embedding_tr[(embedding_tr['lab']==2)]
#embedding_tr = reducer.transform(data_pca_tr)

#embedding_final = np.concatenate((embedding, embedding_tr), axis=0)

colors = oob_all.loc[:,'lab']
palette = np.array(sns.color_palette("hls", 3))
f = plt.figure(figsize=(8, 8))
ax = plt.subplot(aspect='equal')
sc = ax.scatter(embedding[:,0], embedding[:,1], lw=0, s=40,c=palette[colors.astype(np.int)])
for index in range(len(embedding_tr_start)):
    plt.arrow(embedding_tr_start.iloc[index,0], embedding_tr_start.iloc[index,1], 
                  embedding_tr_end.iloc[index,0] - embedding_tr_start.iloc[index,0], 
              embedding_tr_end.iloc[index,1] - embedding_tr_start.iloc[index,1]
                 )

#sc = ax.scatter(embedding[:,0], embedding[:,1], lw=0, s=40)
plt.xlim(-25, 25)
plt.ylim(-25, 25)

ax.axis('tight')
plt.show()

In [ ]:
df_style = copy.deepcopy(input_ltpm_matrix)
df_style['labels'] = copy.deepcopy(input_ltpm_matrix_labs)
df_style['states'] = copy.deepcopy(input_ltpm_matrix_states)

df_style['id'] = df_style.labels.map(str) + "" + df_style.states.map(str)
df_style = df_style.drop('labels', 1)
df_style = df_style.drop('states', 1)
#df_style = df_style[(df_style['id']!='61')]

mean_dict_stats = {}
for index in np.unique(df_style['id']):
    df_style_temp =  copy.deepcopy(df_style[(df_style['id']==index)])
    mean_dict_stats[index] = df_style_temp.mean(axis=0)
#df_style = df_style.drop('id', 1)
mean_dict_stats = pd.DataFrame(mean_dict_stats).T.reset_index(drop = True)
mean_dict_stats = mean_dict_stats.drop('id', 1)


In [ ]:
sim_ends_pd_stats = copy.deepcopy(sim_ends_pd).drop('lab', 1).mean(axis=0)
mean_dict_stats = mean_dict_stats.T.reset_index().T.reset_index(drop=True).iloc[1:]
mean_dict_stats = mean_dict_stats.append(sim_ends_pd_stats,ignore_index=True)
reqd_cols = mean_dict_stats.columns
mean_dict_stats[reqd_cols] = mean_dict_stats[reqd_cols].astype('float64')

In [ ]:
mean_dict_stats.T

In [ ]:
sns.pairplot(mean_dict_stats.T)

In [ ]:
corr_stat_ids = np.concatenate(np.unique(df_style['id']), ['gen'])

In [ ]:
np.unique(df_style['id'])

In [ ]:
df_corr = pd.DataFrame() # Correlation matrix
df_p = pd.DataFrame()  # Matrix of p-values
for x in mean_dict_stats.T.columns:
    for y in mean_dict_stats.T.columns:
        corr = stats.pearsonr(mean_dict_stats.T[x], mean_dict_stats.T[y])
        df_corr.loc[x,y] = corr[0]
        df_p.loc[x,y] = corr[1]

In [ ]:
plt.pcolor(df_corr)
plt.yticks(np.arange(0.5, len(df_corr.index), 1), df_corr.index)
plt.xticks(np.arange(0.5, len(df_corr.columns), 1), df_corr.columns)
plt.show()

In [ ]:
oob = copy.deepcopy(df_filtered_start_oob)
oob2 = copy.deepcopy(df_filtered_end_oob)

oob = oob.assign(lab = 0)

oob2 = oob2.assign(lab = 1)

oob = oob.append(oob2, ignore_index=True)


pca = PCA(n_components=100)
data_pca = pca.fit_transform(oob.loc[:, oob.columns != 'lab'])
#data_pca_tr = pca.transform(df.loc[:, df.columns != 'lab'])

#data_pca_tr = pca.transform(true_starters_pd)

reducer = umap.UMAP()

embedding = reducer.fit_transform(data_pca)
#embedding_tr = reducer.transform(data_pca_tr)

#embedding_final = np.concatenate((embedding, embedding_tr), axis=0)

colors = oob.loc[:,'lab']
palette = np.array(sns.color_palette("hls", 2))
f = plt.figure(figsize=(8, 8))
ax = plt.subplot(aspect='equal')
sc = ax.scatter(embedding[:,0], embedding[:,1], lw=0, s=40,c=palette[colors.astype(np.int)])
#sc = ax.scatter(embedding[:,0], embedding[:,1], lw=0, s=40)
plt.xlim(-25, 25)
plt.ylim(-25, 25)

ax.axis('tight')
plt.show()

In [ ]:
sns.palplot(sns.color_palette("hls", 3))


In [ ]:
pd.DataFrame(sim_ends)


# vsa